In [207]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler,RobustScaler,StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout,Bidirectional, LSTM, SimpleRNN,GRU,Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint


In [208]:
# Load the dataset
df = pd.read_csv('..//dataset//final_dataset_compressed.csv')  # Replace with your dataset path

# Separating features and target
X = df[['Number of Lanes', 'Status of driver', 'Nature of environment', 'Velocity of vehicle', 'Distance between vehicles', 'Breaking capability']]
y = df['Collussion']
# scaler=RobustScaler()
# X_scaled = scaler.fit_transform(X)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
# X_scaled = X

# Reshape input to be 3D [samples, timesteps, features] for LSTM
X_scaled = np.reshape(X_scaled, (X_scaled.shape[0], 1, X_scaled.shape[1]))

# Split into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)


In [209]:
print(X_train.shape, y_train.shape,X_scaled.shape)

(320000, 1, 6) (320000,) (400000, 1, 6)


# Model

In [210]:
def build_deep_rnn_bi_lstm():
    model = Sequential()
    
    # Add multiple SimpleRNN layers for Deep RNN
    model.add(Input(shape=(X_train.shape[1], X_train.shape[2])))
    model.add(SimpleRNN(units=50, return_sequences=True))
    model.add(Dropout(0.1))
    model.add(SimpleRNN(units=50, return_sequences=True))
    model.add(Dropout(0.1))

    # Add multiple Bidirectional LSTM layers
    model.add(Bidirectional(LSTM(units=50, return_sequences=True)))
    model.add(Dropout(0.1))
    model.add(Bidirectional(LSTM(units=50, return_sequences=True)))
    model.add(Dropout(0.1))
    model.add(Bidirectional(LSTM(units=50)))
    model.add(Dropout(0.1))

    # Output layer
    model.add(Dense(units=1, activation='sigmoid'))

    # Compile the model
    model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['mse', 'mae'])

    return model


In [211]:
# Early stopping callback
early_stopping = EarlyStopping(
    monitor='mse',  
    patience=3,   
    restore_best_weights=True, 
    verbose=1
)
# Model checkpoint callback
best_weights_filepath ='..//models//best_model.keras'

model_checkpoint = ModelCheckpoint(
    best_weights_filepath,  
    monitor='val_mse',  
    mode='min',       
    save_best_only=True,   
    verbose=1
)

# Instantiate and train the model
model = build_deep_rnn_bi_lstm()
history = model.fit(X_train, y_train, epochs=5, batch_size=128, validation_split=0.2, verbose=1,   
#  callbacks=[model_checkpoint,early_stopping]
)

Epoch 1/5
2000/2000 ━━━━━━━━━━━━━━━━━━━━ 17s 5ms/step - loss: 0.0926 - mae: 0.0644 - mse: 0.0300 - val_loss: 0.0322 - val_mae: 0.0216 - val_mse: 0.0110
Epoch 2/5
2000/2000 ━━━━━━━━━━━━━━━━━━━━ 10s 5ms/step - loss: 0.0338 - mae: 0.0225 - mse: 0.0113 - val_loss: 0.0313 - val_mae: 0.0215 - val_mse: 0.0107
Epoch 3/5
2000/2000 ━━━━━━━━━━━━━━━━━━━━ 10s 5ms/step - loss: 0.0315 - mae: 0.0211 - mse: 0.0105 - val_loss: 0.0295 - val_mae: 0.0196 - val_mse: 0.0101
Epoch 4/5
2000/2000 ━━━━━━━━━━━━━━━━━━━━ 10s 5ms/step - loss: 0.0304 - mae: 0.0204 - mse: 0.0103 - val_loss: 0.0287 - val_mae: 0.0202 - val_mse: 0.0097
Epoch 5/5
2000/2000 ━━━━━━━━━━━━━━━━━━━━ 10s 5ms/step - loss: 0.0303 - mae: 0.0204 - mse: 0.0102 - val_loss: 0.0291 - val_mae: 0.0196 - val_mse: 0.0100


In [212]:
# import numpy as np

# # Set numpy to display full float numbers
# np.set_printoptions(suppress=True, precision=10)

In [213]:
# results = model.evaluate(X_test, y_test, verbose=1)
# loss, mse, mae = results  # Unpack all the returned metrics
# print(f'Loss: {loss:.4f}')
# print(f'Mean Squared Error (MSE): {mse:.4f}')
# print(f'Mean Absolute Error (MAE): {mae:.4f}')


In [214]:

from sklearn.metrics import mean_squared_error
import numpy as np

# Make predictions
y_pred = model.predict(X_test)

# Calculate MSE
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error (MSE): {mse:.16f}')

# Calculate RMSE
rmse = np.sqrt(mse)
print(f'Root Mean Squared Error (RMSE): {rmse:.16f}')

2500/2500 ━━━━━━━━━━━━━━━━━━━━ 4s 1ms/step
Mean Squared Error (MSE): 0.0099383844062686
Root Mean Squared Error (RMSE): 0.0996914460034992


In [215]:
# import matplotlib.pyplot as plt

# # Visualizing the loss, MSE, and MAE
# plt.figure(figsize=(15, 5))

# # Loss plot
# plt.subplot(1, 3, 1)
# plt.plot(history.history['loss'], label='Train Loss')
# plt.plot(history.history['val_loss'], label='Validation Loss')
# plt.title('Model Loss')
# plt.ylabel('Loss')
# plt.xlabel('Epoch')
# plt.legend(loc='upper right')

# # MSE plot
# plt.subplot(1, 3, 2)
# plt.plot(history.history['mse'], label='Train MSE')
# plt.plot(history.history['val_mse'], label='Validation MSE')
# plt.title('Mean Squared Error')
# plt.ylabel('MSE')
# plt.xlabel('Epoch')
# plt.legend(loc='upper right')

# # MAE plot
# plt.subplot(1, 3, 3)
# plt.plot(history.history['mae'], label='Train MAE')
# plt.plot(history.history['val_mae'], label='Validation MAE')
# plt.title('Mean Absolute Error')
# plt.ylabel('MAE')
# plt.xlabel('Epoch')
# plt.legend(loc='upper right')

# plt.tight_layout()
# plt.show()


In [216]:
#saving the model
# model.save('..//models//deep_rnn_bi_lstm_updated_1.keras')  # Replace with your model path